In [13]:
import cv2
import numpy as np
import tensorflow as tf
from PIL import Image
from crossword_algorithm import find_words

In [14]:
#while True:
    # ret, frame3 = video.read()

    # frame4 = cv2.cvtColor(frame3, cv2.COLOR_BGR2GRAY)
    # #frame = cv2.blur(frame4, (100,1))

    # frame = cv2.adaptiveThreshold(frame4, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 21, 7)

    # cv2.imshow("vid", frame)

    # if cv2.waitKey(4) & 0xFF == ord('q'):
    #     break

# video.release()
# cv2.destroyAllWindows()

In [15]:
image2 = cv2.imread("image_samples/word_search_summer.png")
image = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
#ret, image = cv2.threshold(image, 100, 255, cv2.THRESH_BINARY_INV)
image = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 33, 25)

image = cv2.blur(image, (2,2))

contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

area = 0

for cnt in contours:
    area += cv2.contourArea(cnt)

area = area/len(contours)

letter_grid_coordinates = dict()
y_top, y_bottom = -1,-1

for i in range(len(contours)):
    #cv2.drawContours(image2, cnt, -1, (255,0,0), 7 )
    if cv2.contourArea(contours[i]) < 0.1*area:
        continue
    x, y, w, h = cv2.boundingRect(contours[i])
    #cv2.rectangle(image2, (x - int(w*(0.45)), y - int(h*(0.45))), (x + w + int(w*(0.45)), y + h + int(h*(0.45))), (0, 255, 0), 2)

    pushed = False

    for key in letter_grid_coordinates.keys():
        if y + w//2 in range(key[0], key[1]):
            y_top, y_bottom = y, y+h
            letter_grid_coordinates[key].append((x - int((w)*(0.25)), y - int(h*(0.25)), x + w + int(w*(0.25)), y + h + int(h*(0.25))))
            pushed = True

    if not pushed:
        letter_grid_coordinates[(y, y+w)] = [(x - int(w*(0.25)), y - int(h*(0.25)), x + w + int(w*(0.25)), y + h + int(h*(0.25)))]
    
    #cv2.rectangle(image2, (x - int(w*(0.45)), y - int(h*(0.45))), (x + w + int(w*(0.45)), y + h + int(h*(0.45))), (0, 255, 0), 2)
    

In [16]:
# video = cv2.VideoCapture(0)

# while True:
#     r, image2 = video.read()
#     image = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
#     #ret, image = cv2.threshold(image, 100, 255, cv2.THRESH_BINARY_INV)
#     image = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 33, 25)

#     image = cv2.blur(image, (2,2))

#     contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

#     area = 0

#     for cnt in contours:
#         area += cv2.contourArea(cnt)

#     area = area/len(contours)

#     for i in range(len(contours)):
#         #cv2.drawContours(image2, cnt, -1, (255,0,0), 7 )
#         if cv2.contourArea(contours[i]) < 0.1*area:
#             continue
#         x, y, w, h = cv2.boundingRect(contours[i])
#         cv2.rectangle(image2, (x - int(w*(0.45)), y - int(h*(0.45))), (x + w + int(w*(0.45)), y + h + int(h*(0.45))), (0, 255, 0), 2)

#     cv2.imshow("hsf", image2)
#     cv2.imshow("hsf2", image)

#     cv2.waitKey(80)

In [17]:
model = tf.keras.models.load_model("font_identifier.keras")

In [18]:
cv2.imshow("hsf", image2)
cv2.imshow("hsf2", image)
cv2.waitKey(1000)
cv2.destroyAllWindows()

crossword = list()

for i in letter_grid_coordinates.keys():
    letter_grid_coordinates[i] = sorted(letter_grid_coordinates[i], key=lambda x: x[0])
    #print(letter_grid_coordinates[i])

for i in reversed(letter_grid_coordinates.values()):
    x = list()
    for j in i:
        x1, y1, x2, y2 = j
        img = image[y1:y2, x1:x2]
        img = Image.fromarray(img)
        img.thumbnail((28, 28), Image.Resampling.LANCZOS)

        pillow_image = Image.new("L", (28, 28), 0)
        pillow_image.paste(img, ((28 - img.size[0]) // 2, (28 - img.size[1]) // 2))

        img = np.array(pillow_image).reshape((1,28,28,1))
        img = img/255.0
        prediction = model.predict(img)
        x.append(chr(np.argmax(prediction) + 65))



    crossword.append(x)

# j = list(letter_grid_coordinates.values())[8][17]

# x1, y1, x2, y2 = j
# img = image[y1:y2, x1:x2]
# img = cv2.resize(img, (28, 28))
# cv2.imshow(str(x1), img)
# img = img.reshape((-1,28,28,1))
# img = img/255.0
# prediction = model.predict(img)
# print(chr(np.argmax(prediction) + 65))

# for i in letter_grid_coordinates.keys():
#     print(len(letter_grid_coordinates[i]) , "elements in", i)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━

In [19]:
print(crossword)

[['G', 'A', 'R', 'D', 'E', 'N', 'Y', 'S', 'M', 'B', 'M', 'T', 'O', 'L', 'S'], ['N', 'I', 'A', 'R', 'M', 'E', 'L', 'E', 'S', 'E', 'E', 'R', 'W', 'O', 'U'], ['U', 'C', 'I', 'N', 'C', 'I', 'P', 'E', 'A', 'S', 'I', 'A', 'M', 'S', 'N'], ['L', 'M', 'L', 'I', 'N', 'O', 'I', 'T', 'A', 'C', 'A', 'V', 'R', 'P', 'S'], ['P', 'O', 'P', 'S', 'I', 'C', 'L', 'E', 'O', 'L', 'D', 'E', 'K', 'I', 'H'], ['U', 'I', 'P', 'E', 'C', 'A', 'M', 'P', 'I', 'O', 'O', 'L', 'F', 'C', 'I'], ['Y', 'T', 'N', 'S', 'R', 'E', 'T', 'L', 'M', 'D', 'N', 'T', 'R', 'E', 'N'], ['P', 'W', 'N', 'S', 'O', 'E', 'P', 'A', 'T', 'G', 'N', 'E', 'D', 'X', 'E'], ['A', 'U', 'G', 'R', 'I', 'A', 'D', 'Y', 'B', 'O', 'A', 'Y', 'A', 'T', 'E'], ['R', 'L', 'J', 'U', 'L', 'Y', 'G', 'R', 'S', 'A', 'S', 'O', 'U', 'P', 'A'], ['K', 'E', 'F', 'G', 'L', 'C', 'E', 'A', 'T', 'H', 'I', 'R', 'G', 'A', 'S'], ['B', 'A', 'L', 'Y', 'P', 'A', 'E', 'N', 'O', 'C', 'R', 'E', 'U', 'R', 'D'], ['S', 'H', 'O', 'R', 'T', 'S', 'O', 'O', 'H', 'A', 'E', 'H', 'S', 'H', 'R']

In [20]:
for i in crossword:
    print(' '.join(i))

G A R D E N Y S M B M T O L S
N I A R M E L E S E E R W O U
U C I N C I P E A S I A M S N
L M L I N O I T A C A V R P S
P O P S I C L E O L D E K I H
U I P E C A M P I O O L F C I
Y T N S R E T L M D N T R E N
P W N S O E P A T G N E D X E
A U G R I A D Y B O A Y A T E
R L J U L Y G R S A S O U P A
K E F G L C E A T H I R G A S
B A L Y P A E N O C R E U R D
S H O R T S O O H A E H S H R
U Y P E M N O S A E S S T D E
C H E T A M C L Y B G M R E S
X L A O V Z R C O J B R I N S
F O S E M A E R C E C I O W L
B L O S S U M M E R W B M O S


In [21]:
print(find_words(crossword, ['GAR', 'MV']))

[((0, 0), (0, 2)), ((2, 12), (3, 11))]
